In [ ]:
# Final Hotel Project

import pyodbc
class Data_Base:
    def __init__(self):
        self.conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=LAPTOP-NUB4LAN9\SQLEXPRESS;"
            "DATABASE=hotel_project;"
            "Trusted_Connection=yes;"
        )
        self.cursor = self.conn.cursor()
        print("Database Connected")
        

class Hotel(Data_Base):
    def __init__(self):
        super().__init__()
        self.menu = {}
        self.load_menu()

    def load_menu(self):
        self.menu.clear()
        self.cursor.execute("SELECT menu_name,price FROM menu")
        for name,price in self.cursor.fetchall():
            self.menu[name]=price

    def show_menu(self):
        print("\n------MENU------")
        for item,price in self.menu.items():
            print(item,":",price)

    def add_item(self):
        name=input("Enter item:")
        price=int(input("Enter price:"))

        self.cursor.execute("INSERT INTO menu VALUES (?,?)",(name,price))
        self.conn.commit()
        print("Item added")


class Order:
    def __init__(self,hotel):
        self.hotel=hotel
        self.items={}
    
    def take(self):
        self.name=input("Customer name:").strip()
        while True:
            item=input("Enter the food item you want :")
            if item not in self.hotel.menu:
                print("The Item is not available - try again ")
                continue
            qty=int(input("Quantity:"))
            self.items[item]=self.items.get(item,0)+qty
            more=input("want more? yes or no: ").lower()
            if more in ["n","no"]:
                break

    def save(self):
        total=0  
        for item,qty in self.items.items():
            price=self.hotel.menu[item]*qty
            total+=price
            self.hotel.cursor.execute(
                "INSERT INTO orders(cust_name,menu_name,quantity,price) VALUES(?,?,?,?)",
                (self.name,item,qty,price)
            )
        self.hotel.cursor.execute("INSERT INTO bill VALUES (?,?)",(self.name,total) )
        self.hotel.conn.commit()

    def print_bill(self):
        filename = f"{self.name}_bill.txt"
        import os
        print("Saving at:", os.path.abspath(filename))
        with open(filename, "w") as file:
            print("\nCustomer:", self.name)
            print("-----------------------")

            file.write("Customer: "+self.name+"\n")
            file.write("-----------------------\n")

            total = 0

            for item,qty in self.items.items():
                price=self.hotel.menu[item]*qty
                line=f"{item}   {qty}   {price}"
                print(line)
                file.write(line+"\n")
                total+=price

            print("-----------------------")
            print("TOTAL =",total)

            file.write("-----------------------\n")
            file.write("TOTAL = "+str(total))

        print("Bill saved to file:",filename)


hotel=Hotel()
while True:
    print("""
1 Show Menu
2 Add Item
3 Take Order
4 Exit
""")
    
    ohoh = input("Enter your Option from the given options (1-4):")
    if ohoh == "1":
        hotel.show_menu()
    elif ohoh == "2":
        hotel.add_item()
        hotel.load_menu()
    elif ohoh == "3":
        o=Order(hotel)
        o.take()
        o.save()
        o.print_bill()
    elif ohoh == "4":
        break

